# 参考程度に
- やったこと
- disclosureNumberをdfに追加
- dfのindexに存在するNone，indexを削除
- col_dicを作成し，それ以外を削除（これの関数については使用しなくても良いです）

# 作成したdfについて
- thisdfが今季の結果を表したdf
- predfが前期の結果を表したdf

- predfの上４つをthisdfに合わせると，八文字くんの作成していたdfになると思います．

# その他
- 曖昧なんですが，決算短信には，前期の結果がないものがあった気がします．
もしあったらエラーを吐くと思うので，その際はif文などを追加するといいと思います．

In [40]:
# モジュールのインポート
import pandas as pd
import os
import tqdm

In [41]:
path='../new2/xbrl_df_cp932'

In [42]:
col_dic={
    "売上高":"Sales",
    "営業利益":"Operating",
    "事業利益":"Business",
    "経常利益":"Ordinary",
    "親会社株主に帰属する当期純利益":"ParentNetIncome",
    "親会社株主に帰属する四半期純利益":"QuarterParentNetIncome",
    "1株当たり当期純利益":"OneNetIncome",
    "1株当たり四半期純利益":"QuarterOneNetIncome"
}

In [43]:
path='../new2/xbrl_df_cp932'
# xbrl_dfフォルダの中身の名前を取得
folderlist = os.listdir(path)

In [44]:
com_list=[]
for i in folderlist:
    if len(i)==5:
        com_list.append(i)

In [45]:
folderlist=com_list

In [46]:
# xbrl_df/13010の中身のファイル名を取得
path2=path+"/"+"{}".format(folderlist[0])
filelist = os.listdir(path2)
filelist=sorted(filelist)

In [47]:
# 業績である_1だけを抽出
acvlist = []
for i in range(len(filelist)):
    if '_1' in filelist[i]:
        acvlist.append(filelist[i])
acvlist=sorted(acvlist)

In [48]:
acvlist

['20180329400196_1.csv',
 '20180710479571_1.csv',
 '20180927411445_1.csv',
 '20190104456625_1.csv',
 '20190329497919_1.csv',
 '20190711470789_1.csv',
 '20190927402419_1.csv',
 '20200107444140_1.csv',
 '20200330487161_1.csv',
 '20200710461021_1.csv',
 '20200929497072_1.csv',
 '20201229440153_1.csv',
 '20210326485749_1.csv',
 '20210713466033_1.csv',
 '20210928403572_1.csv',
 '20211230563188_1.csv',
 '20220329513191_1.csv']

In [49]:
#col_dicにあるデータが基本的には必要になる
#そのため，col_dicにないものは削除をする
#順序としては，columnを逆からみて，col_dicに存在するcolumn出現したら，そこまでをdataframeとする
def recreate_df(df):
    index=df.loc["index"]
    for i in range(len(index)-1,-1,-1):
        if index[i] in col_dic:
            break
    return(df.iloc[:,:i+1])

In [50]:
#Noneとindexを削除
def re_index(df):
    column = [df.loc["index"][i]+df.loc["None"][i] for i in range(0,len(df.columns))]
    df.columns=column
    index=[df.index[i] for i in range(1,len(df.index)-1)]
    #index.insert(0,"index")
    df=df.reindex(index=index)
    return(df)

In [51]:
#disclosureNumberをdfに追加
def create_number(df,num):
    number_list=[]
    for i in range(0,len(df)):
        number_list.append(num[:14])
    df["number"] = number_list
    return(df)

In [52]:
#予測を追加
def create_predict(df,text,flag):
    pre_list=[]
    term_list=[]
    for i in range(0,len(df)):
        pre_list.append(flag)
        term_list.append(pre_text)

    df["prediction"]=pre_list
    df["pre_term"]=term_list
    return(df)

In [53]:
i=0

In [15]:
path3=path2+"/"+"{}".format(acvlist[i])
df = pd.read_csv(path3, index_col=0,encoding="cp932")
df=recreate_df(df)
df=re_index(df)
df=create_number(df,acvlist[i])
df

,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する四半期純利益百万円,親会社株主に帰属する四半期純利益％,1株当たり四半期純利益円銭,number
Unnamed: 0,,,,,,,,,,
30年5月期第3四半期,41552,None,-45,None,13,None,-57,None,-22.71,20180329400196
29年5月期第3四半期,None,None,None,None,None,None,None,None,None,20180329400196


In [15]:
for i in range(0,len(acvlist)):
    path3=path2+"/"+"{}".format(acvlist[i])
    df = pd.read_csv(path3, index_col=0,encoding="cp932")
    df=recreate_df(df)
    df=re_index(df)
    df=create_number(df,acvlist[i])
    #今季の結果のdataframe
    adf = df.drop(index=df.index[[1]])
    #前期の結果のdataframe
    bdf = df.drop(index=df.index[[0]])
    if i==0:
        thisdf = adf
        predf = bdf
        continue
    else:
        thisdf = pd.concat([thisdf,adf])
        predf = pd.concat([predf,bdf])
    
    #binidf = pd.concat([binidf,bdf])
    
#bdropdf = binidf.drop_duplicates(keep='first')

In [29]:
thisdf

,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する当期純利益百万円,親会社株主に帰属する当期純利益％,1株当たり当期純利益円銭,number,親会社株主に帰属する四半期純利益百万円,親会社株主に帰属する四半期純利益％,1株当たり四半期純利益円銭
Unnamed: 0,,,,,,,,,,,,,
29年6月期,4516,13.3,186,―,239,―,134,―,21.40,20170817458065,NaN,NaN,NaN
30年6月期第1四半期,1276,47.3,82,―,81,―,NaN,NaN,NaN,20171108411889,40,―,6.37
30年6月期第2四半期,2783,33.8,263,502.6,274,181.1,NaN,NaN,NaN,20180202462903,163,121.4,259.57
30年6月期第3四半期,4164,22.2,502,175.2,505,114.9,NaN,NaN,NaN,20180507427641,301,86.1,478.69
30年6月期,5532,22.5,549,194.6,587,145.7,321,138.6,510.68,20180804491900,NaN,NaN,NaN
30年6月期第2四半期,2770,33.1,249,468.7,256,163.0,NaN,NaN,NaN,20180913406805,151,104.5,239.73
30年6月期第3四半期,4151,21.8,485,166.3,485,106.3,NaN,NaN,NaN,20180914407068,287,77.1,455.47
31年6月期第1四半期,1261,-1.1,150,83.1,175,114.9,NaN,NaN,NaN,20181025422848,111,177.1,176.48
31年6月期第2四半期,2909,5.0,359,44.5,403,57.2,NaN,NaN,NaN,20190130465661,255,69.1,405.48


In [30]:
predf

,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する当期純利益百万円,親会社株主に帰属する当期純利益％,1株当たり当期純利益円銭,number,親会社株主に帰属する四半期純利益百万円,親会社株主に帰属する四半期純利益％,1株当たり四半期純利益円銭
Unnamed: 0,,,,,,,,,,,,,
28年6月期,3986,-9.5,-80,―,-131,―,-175,―,-27.77,20170817458065,NaN,NaN,NaN
29年6月期第1四半期,866,2.0,-31,―,-33,―,NaN,NaN,NaN,20171108411889,-29,―,-4.70
29年6月期第2四半期,2081,6.8,43,―,97,―,NaN,NaN,NaN,20180202462903,73,―,117.25
29年6月期第3四半期,3409,10.8,182,―,235,―,NaN,NaN,NaN,20180507427641,162,―,257.18
29年6月期,4516,13.3,186,―,239,―,134,―,214.00,20180804491900,NaN,NaN,NaN
29年6月期第2四半期,2081,6.8,43,―,97,―,NaN,NaN,NaN,20180913406805,73,―,117.25
29年6月期第3四半期,3409,10.8,182,―,235,―,NaN,NaN,NaN,20180914407068,162,―,257.18
30年6月期第1四半期,1276,47.3,82,―,81,―,NaN,NaN,NaN,20181025422848,40,―,63.69
30年6月期第2四半期,2770,33.1,249,468.7,256,163.0,NaN,NaN,NaN,20190130465661,151,104.5,239.73


In [54]:
import tqdm

In [55]:
#全企業

In [56]:
err_list=[]
for com in tqdm.tqdm(folderlist):
    path2=path+"/"+"{}".format(com)
    filelist = os.listdir(path2)
    filelist=sorted(filelist)
    # 業績である_1だけを抽出
    acvlist = []
    for i in range(len(filelist)):
        if '_1' in filelist[i]:
            acvlist.append(filelist[i])
    acvlist=sorted(acvlist)

    try:
        for i in range(0,len(acvlist)):
            path3=path2+"/"+"{}".format(acvlist[i])
            df = pd.read_csv(path3, index_col=0,encoding="cp932")
            df=recreate_df(df)
            df=re_index(df)
            df=create_number(df,acvlist[i])
            #今季の結果のdataframe
            adf = df.drop(index=df.index[[1]])
            #前期の結果のdataframe
            bdf = df.drop(index=df.index[[0]])
            if i==0:
                thisdf = adf
                predf = bdf
                continue
            else:
                thisdf = pd.concat([thisdf,adf])
                predf = pd.concat([predf,bdf])
        
        #thisdf.to_csv("../new2/df_sum/this/{}.csv".format(com),encoding="cp932")
        #predf.to_csv("../new2/df_sum/pre_term/{}.csv".format(com),encoding="cp932")
    except:
        err_list.append(com)

100%|███████████████████████████████████████| 3225/3225 [02:12<00:00, 24.32it/s]


In [58]:
len(err_list)

15

In [57]:
err_list

['74850',
 '83030',
 '31680',
 '65880',
 '32710',
 '49670',
 '31330',
 '73390',
 '47220',
 '94680',
 '67860',
 '29820',
 '39990',
 '56030',
 '35530']

In [264]:
company.loc[35530]

name                                 共和レザー株式会社
class                                       化学
SettlementDate                           3月31日
end                    2022年３月期 決算短信〔日本基準〕（連結）
start             平成30年３月期 第１四半期決算短信〔日本基準〕（連結）
Name: 35530, dtype: object

In [ ]:
74850
94680
67860
35530

# エラーの処理

In [ ]:
#83030
#31680
#65880
#32710
#49670
#31330
#73390
#47220
#67860
#29820
#39990
#56030

In [148]:
com=err_list[15]
path2=path+"/"+"{}".format(com)
filelist = os.listdir(path2)
filelist=sorted(filelist)
# 業績である_1だけを抽出
acvlist = []
for i in range(len(filelist)):
    if '_1' in filelist[i]:
        acvlist.append(filelist[i])
acvlist=sorted(acvlist)
for i in range(0,len(acvlist)):
    path3=path2+"/"+"{}".format(acvlist[i])
    df = pd.read_csv(path3, index_col=0,encoding="cp932")
    
    if "index" in list(df.index):
        df=recreate_df(df)
        df=re_index(df)
        df=create_number(df,acvlist[i])
    else:
        df.columns=[k for k in range(0,int(df.shape[1]))]
        df=create_number(df,acvlist[i])
        df.columns=[kari_col[k] for k in range(0,len(df.columns))]
        index=[df.index[i] for i in range(1,len(df.index))]
        df=df.reindex(index=index)
    
    #今季の結果のdataframe
    adf = df.drop(index=df.index[[1]])
    #前期の結果のdataframe
    bdf = df.drop(index=df.index[[0]])
    if i==0:
        thisdf = adf
        predf = bdf
        continue
    else:
        thisdf = pd.concat([thisdf,adf])
        predf = pd.concat([predf,bdf])
        
        kari_col=list(thisdf.columns)
        
thisdf.to_csv("../new2/df_sum/err/this/{}.csv".format(com),encoding="cp932")
predf.to_csv("../new2/df_sum/err/pre_term/{}.csv".format(com),encoding="cp932")

IndexError: list index out of range

In [145]:
path3

'../new2/xbrl_df_cp932/35530/20220510537187_1.csv'

In [146]:
df

,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する当期純利益百万円,親会社株主に帰属する当期純利益％,1株当たり当期純利益円銭,number
Unnamed: 0,,,,,,,,,,
2022年3月期,47074,14.3,1819,15.8,2285,19.8,1684,16.8,68.85,20220510537187
2021年3月期,41182,-4.1,1571,-23.4,1907,-13.3,1442,12.9,58.86,20220510537187


In [147]:
thisdf

,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する四半期純利益百万円,親会社株主に帰属する四半期純利益％,1株当たり四半期純利益円銭,number,親会社株主に帰属する当期純利益百万円,親会社株主に帰属する当期純利益％,1株当たり当期純利益円銭
Unnamed: 0,,,,,,,,,,,,,
30年3月期第1四半期,11465,0.1,700,-18.5,767,-1.8,482,-4.4,19.69,20170728442442,NaN,NaN,NaN
30年3月期第2四半期,23106,3.7,1494,-9.3,1594,1.9,995,0.1,40.65,20171030403552,NaN,NaN,NaN
30年3月期第3四半期,35459,4.9,2605,-8.1,2745,-6.2,1800,-3.1,73.51,20180206465463,NaN,NaN,NaN
30年3月期,49003,4.8,3247,-13.9,3349,-11.9,NaN,NaN,NaN,20180508428944,2399,-6.2,97.95
2019年3月期第1四半期,11877,3.6,611,-12.6,647,-15.7,436,-9.5,17.82,20180730488215,NaN,NaN,NaN
2019年3月期第2四半期,22978,-0.6,887,-40.6,953,-40.2,629,-36.8,25.71,20181106430966,NaN,NaN,NaN
2019年3月期第3四半期,34980,-1.4,1454,-44.2,1522,-44.6,1055,-41.4,43.07,20190205470612,NaN,NaN,NaN
2019年3月期,45682,-6.8,1949,-40.0,2043,-39.0,NaN,NaN,NaN,20190509418764,1498,-37.6,61.16
2020年3月期第1四半期,10514,-11.5,295,-51.7,264,-59.2,198,-54.6,8.09,20190730478513,NaN,NaN,NaN


## 以下予測の表


In [20]:
import tqdm

In [224]:
non_list=[]
for i in tqdm.tqdm(range(0,len(folderlist))):
#for i in range(0,3):
    # xbrl_df/13010の中身のファイル名を取得
    path2=path+"/"+"{}".format(folderlist[i])
    filelist = os.listdir(path2)
    filelist=sorted(filelist)

    # 業績である_1だけを抽出
    acvlist = []
    # 業績であるtxtだけを抽出
    textlist = []
    for j in range(len(filelist)):
        if '_2' in filelist[j] or 'NoPre' in filelist[j]:
            acvlist.append(filelist[j])
        
        elif ('txt' in filelist[j]) and (len(filelist[j]) >10 ):
            textlist.append(filelist[j])
    acvlist=sorted(acvlist)
    textlist=sorted(textlist)
    try:
        
        for k in range(0,len(acvlist)):
            txt_path=path2+"/"+"{}".format(textlist[k])
            with open (txt_path,'r') as t:
                pre_text=t.read()
            if "csv" in acvlist[k]:
                path3=path2+"/"+"{}".format(acvlist[k])
                df = pd.read_csv(path3, index_col=0,encoding="cp932")
                df=recreate_df(df)

                if k==0:
                    df=re_index(df)
                    kari_col=[df.columns[n] for n in range(0,len(df.columns))]

                else:
                    if str(df.loc["None"][0])=="nan":
                           df.columns=kari_col
                    else:
                           df=re_index(df)

                flag=True
                df=create_number(df,acvlist[k])
                df = create_predict(df,pre_text,flag)

            else:
                flag=False
                None_list=[None for n in range(0,len(df.columns))]
                column = [df.columns[n] for n in range(0,len(df.columns))]
                df= pd.DataFrame([None_list],index=["通期"],columns=column)
                df=create_number(df,acvlist[k])
                df = create_predict(df,pre_text,flag)

            if k==0:
                newdf = df
            else:
                newdf = pd.concat([newdf,df])
        
        #newdf.to_csv("../new2/df_sum/prediction/{}.csv".format(folderlist[i]),encoding="cp932")
                
    except:
        non_list.append(folderlist[i])

100%|███████████████████████████████████████| 3225/3225 [01:48<00:00, 29.79it/s]


In [225]:
len(non_list)

216

In [152]:
sorted(non_list)

['18480',
 '18900',
 '18920',
 '19540',
 '21380',
 '21390',
 '21810',
 '21830',
 '21910',
 '22160',
 '23700',
 '23760',
 '23780',
 '23890',
 '24060',
 '24370',
 '24400',
 '26070',
 '26660',
 '27670',
 '29020',
 '29820',
 '30420',
 '30450',
 '30570',
 '30690',
 '30700',
 '30730',
 '30850',
 '30890',
 '31130',
 '31570',
 '31680',
 '31820',
 '31950',
 '31960',
 '32450',
 '32520',
 '32710',
 '33280',
 '33500',
 '33580',
 '34170',
 '34790',
 '34860',
 '35410',
 '35690',
 '35800',
 '36070',
 '36220',
 '36240',
 '36320',
 '36390',
 '36550',
 '36640',
 '36650',
 '36720',
 '36780',
 '37530',
 '37600',
 '37650',
 '37930',
 '38080',
 '38470',
 '38540',
 '39030',
 '39090',
 '39110',
 '39140',
 '39220',
 '39230',
 '39270',
 '39280',
 '39940',
 '39990',
 '40420',
 '40630',
 '43850',
 '43870',
 '44360',
 '44480',
 '44780',
 '45630',
 '45640',
 '45820',
 '45840',
 '46610',
 '46710',
 '46860',
 '47220',
 '47650',
 '48000',
 '48190',
 '48830',
 '49110',
 '49180',
 '49670',
 '49870',
 '51220',
 '51940',


In [223]:
#エラーを抜いた企業リストの作成
company = pd.read_csv("../TDNet_full_data/new_company_list.csv",index_col=0)

In [265]:
pop_list=[74850,94680,67860,35530]

In [229]:
pop_list=[]
for i in range(0,len(non_list)):
    if int(non_list[i]) in list(company.index):
        pop_list.append(int(non_list[i]))

In [266]:
new_index=list(company.index)
pop_list=sorted(pop_list)

In [267]:
j=0
i_list=[]
for cnt,i in enumerate(new_index):

    if j!= len(pop_list) and i ==pop_list[j]:
        i_list.append(cnt)
        
        j+=1

In [268]:
for i in i_list[::-1]:
        #print(i)
        new_index.pop(i)

In [269]:
len(new_index)

2782

In [270]:
company=company.reindex(index=new_index)

In [271]:
company

,name,class,SettlementDate,end,start
code,,,,,
13010,株式会社極洋,水産・農林業,3月31日,2022年3月期 決算短信〔日本基準〕(連結),平成30年3月期 第1四半期決算短信〔日本基準〕(連結)
13320,日本水産株式会社,水産・農林業,3月31日,2022年３月期 決算短信 〔日本基準〕（連結）,平成30年3月期第1四半期決算短信〔日本基準〕（連結）
13330,マルハニチロ株式会社,水産・農林業,3月31日,2022年３月期 決算短信〔日本基準〕（連結）,平成30年３月期 第１四半期決算短信〔日本基準〕（連結）
13790,ホクト株式会社,水産・農林業,3月31日,2022年３月期 決算短信〔日本基準〕（連結）,平成30年３月期 第１四半期決算短信〔日本基準〕(連結)
13800,株式会社秋川牧園,水産・農林業,3月31日,2022年３月期 決算短信〔日本基準〕（連結）,平成30年３月期 第１四半期決算短信〔日本基準〕（連結）
...,...,...,...,...,...
99930,株式会社ヤマザワ,小売業,2月28日,2022年２月期 決算短信〔日本基準〕（連結）,平成30年２月期 第１四半期決算短信〔日本基準〕（連結）
99940,株式会社やまや,小売業,3月31日,2022年３月期 決算短信〔日本基準〕(連結),平成30年3月期 第1四半期決算短信〔日本基準〕（連結）
99950,株式会社グローセル,卸売業,3月31日,令和４年３月期 決算短信〔日本基準〕(連結),平成30年3月期第1四半期決算短信〔日本基準〕（連結）


In [272]:
company.to_csv("../TDNet_full_data/kari_company_list.csv")

In [273]:
#エラーを抜いた企業リストの作成
company = pd.read_csv("../TDNet_full_data/kari_company_list.csv",index_col=0)

In [274]:
company

,name,class,SettlementDate,end,start
code,,,,,
13010,株式会社極洋,水産・農林業,3月31日,2022年3月期 決算短信〔日本基準〕(連結),平成30年3月期 第1四半期決算短信〔日本基準〕(連結)
13320,日本水産株式会社,水産・農林業,3月31日,2022年３月期 決算短信 〔日本基準〕（連結）,平成30年3月期第1四半期決算短信〔日本基準〕（連結）
13330,マルハニチロ株式会社,水産・農林業,3月31日,2022年３月期 決算短信〔日本基準〕（連結）,平成30年３月期 第１四半期決算短信〔日本基準〕（連結）
13790,ホクト株式会社,水産・農林業,3月31日,2022年３月期 決算短信〔日本基準〕（連結）,平成30年３月期 第１四半期決算短信〔日本基準〕(連結)
13800,株式会社秋川牧園,水産・農林業,3月31日,2022年３月期 決算短信〔日本基準〕（連結）,平成30年３月期 第１四半期決算短信〔日本基準〕（連結）
...,...,...,...,...,...
99930,株式会社ヤマザワ,小売業,2月28日,2022年２月期 決算短信〔日本基準〕（連結）,平成30年２月期 第１四半期決算短信〔日本基準〕（連結）
99940,株式会社やまや,小売業,3月31日,2022年３月期 決算短信〔日本基準〕(連結),平成30年3月期 第1四半期決算短信〔日本基準〕（連結）
99950,株式会社グローセル,卸売業,3月31日,令和４年３月期 決算短信〔日本基準〕(連結),平成30年3月期第1四半期決算短信〔日本基準〕（連結）


# エラーの処理

In [154]:
company = pd.read_csv("../TDNet_full_data/new_company_list.csv",index_col=0)

In [160]:
in_list=[]
for i in range(0,len(non_list)):
    if int(non_list[i]) in list(company.index):
        in_list.append(non_list[i])

In [162]:
len(in_list)

186

In [169]:
in_list=sorted(in_list)

In [ ]:
if "index" in list(df.index):
        df=recreate_df(df)
        df=re_index(df)
        df=create_number(df,acvlist[i])
    else:
        df.columns=[k for k in range(0,int(df.shape[1]))]
        df=create_number(df,acvlist[i])
        df.columns=[kari_col[k] for k in range(0,len(df.columns))]
        index=[df.index[i] for i in range(1,len(df.index))]
        df=df.reindex(index=index)

In [220]:
#non_list=[]
non_list2=[]
for i in tqdm.tqdm(range(3,len(non_list))):
#for i in range(0,3):
#i=3
    # xbrl_df/13010の中身のファイル名を取得
    path2=path+"/"+"{}".format(non_list[i])
    filelist = os.listdir(path2)
    filelist=sorted(filelist)

    # 業績である_1だけを抽出
    acvlist = []
    # 業績であるtxtだけを抽出
    textlist = []
    for j in range(len(filelist)):
        if '_2' in filelist[j] or 'NoPre' in filelist[j]:
            acvlist.append(filelist[j])

        elif ('txt' in filelist[j]) and (len(filelist[j]) >10 ):
            textlist.append(filelist[j])
    acvlist=sorted(acvlist)
    textlist=sorted(textlist)
    try:
        for k in range(0,len(acvlist)):
            txt_path=path2+"/"+"{}".format(textlist[k])
            with open (txt_path,'r') as t:
                pre_text=t.read()
            if "csv" in acvlist[k]:
                path3=path2+"/"+"{}".format(acvlist[k])
                df = pd.read_csv(path3, index_col=0,encoding="cp932")
                if "index" in list(df.index):
                    df=recreate_df(df)
                    df=re_index(df)
                    df=create_number(df,acvlist[i])
                else:
                    df.columns=[k for k in range(0,int(df.shape[1]))]
                    df=create_number(df,acvlist[i])
                    df.columns=[kari_col[k] for k in range(0,len(df.columns))]
                    index=[df.index[i] for i in range(1,len(df.index))]
                    df=df.reindex(index=index)

                flag=True
                df=create_number(df,acvlist[k])
                df = create_predict(df,pre_text,flag)

            else:
                flag=False
                None_list=[None for n in range(0,len(df.columns))]
                column = [df.columns[n] for n in range(0,len(df.columns))]
                df= pd.DataFrame([None_list],index=["通期"],columns=column)
                df=create_number(df,acvlist[k])
                df = create_predict(df,pre_text,flag)

            if k==0:
                newdf = df
                kari_col=list(newdf.columns)
            else:
                newdf = pd.concat([newdf,df])
                kari_col=list(newdf.columns)
                
        newdf.to_csv("../new2/df_sum/err/prediction/{}.csv".format(non_list[i]),encoding="cp932")
    
    except:
        non_list2.append(non_list[i])

100%|████████████████████████████████████████| 164/164 [00:00<00:00, 206.25it/s]


In [213]:
df

,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する当期純利益百万円,親会社株主に帰属する当期純利益％,1株当たり当期純利益円銭,number,prediction,pre_term
Unnamed: 0,,,,,,,,,,,,
通期,32628,68.6,524,―,903,―,522,―,17.80,20220414521708,True,2023年2月期の連結業績予想


In [215]:
newdf

,売上高百万円,売上高%,営業利益百万円,営業利益%,経常利益百万円,経常利益%,親会社株主に帰属する当期純利益百万円,親会社株主に帰属する当期純利益%,1株当たり当期純利益円銭,number,prediction,pre_term,売上高％,営業利益％,経常利益％,親会社株主に帰属する当期純利益％
第2四半期(累計),20000,33.6,1000,76.8,950,77.7,450,58.2,59.07,20170714431956,True,平成30年2月期の連結業績予想,NaN,NaN,NaN,NaN
通期,44500,45.9,1800,9.7,1700,18.4,700,7.9,91.98,20170714431956,True,平成30年2月期の連結業績予想,NaN,NaN,NaN,NaN
通期,44500,45.9,1800,9.7,1700,18.4,700,7.9,91.98,20171013489999,True,平成30年2月期の連結業績予想,NaN,NaN,NaN,NaN
通期,45000,47.5,2300,40.1,2300,60.2,1100,69.6,144.14,20180112449903,True,平成30年2月期の連結業績予想,NaN,NaN,NaN,NaN
第2四半期(累計),25855,24.0,1247,-22.4,1286,-20.2,770,-29.3,100.72,20180413411087,True,平成31年2月期の連結業績予想,NaN,NaN,NaN,NaN
通期,51078,13.3,2529,14.7,2597,16.8,1402,38.8,183.30,20180413411087,True,平成31年2月期の連結業績予想,NaN,NaN,NaN,NaN
第2四半期(累計),25855,24.0,1350,-16.0,1350,-16.3,1090,-0.0,142.45,20180713481019,True,2019年2月期の連結業績予想,NaN,NaN,NaN,NaN
通期,51078,13.3,2529,14.7,2597,16.8,1402,38.8,183.30,20180713481019,True,2019年2月期の連結業績予想,NaN,NaN,NaN,NaN
通期,51078,13.3,2529,14.7,2597,16.8,1402,38.8,183.30,20181012417018,True,2019年2月期の連結業績予想,NaN,NaN,NaN,NaN
通期,51078,13.3,2529,14.7,2597,16.8,1402,38.8,183.30,20190111459126,True,2019年2月期の連結業績予想,NaN,NaN,NaN,NaN


In [221]:
len(non_list2)

149

In [222]:
non_list2

['30700',
 '34170',
 '34860',
 '35690',
 '35800',
 '36070',
 '36320',
 '36390',
 '36640',
 '36720',
 '36780',
 '37530',
 '37600',
 '37650',
 '37930',
 '38080',
 '38470',
 '38540',
 '39030',
 '39090',
 '39140',
 '39220',
 '39230',
 '39280',
 '39940',
 '39990',
 '40420',
 '40630',
 '43850',
 '43870',
 '44360',
 '44780',
 '45630',
 '45640',
 '45840',
 '46610',
 '46710',
 '46860',
 '47220',
 '47650',
 '48000',
 '48190',
 '48830',
 '49670',
 '49870',
 '51220',
 '51940',
 '53100',
 '53430',
 '53550',
 '54400',
 '54450',
 '56030',
 '56580',
 '56600',
 '56980',
 '59070',
 '59560',
 '59660',
 '60310',
 '60350',
 '60470',
 '60540',
 '60630',
 '60670',
 '60900',
 '61030',
 '61450',
 '61460',
 '61570',
 '61820',
 '62930',
 '63340',
 '63610',
 '64180',
 '64280',
 '64800',
 '64940',
 '64980',
 '65500',
 '65510',
 '65620',
 '65700',
 '65770',
 '65880',
 '66270',
 '66620',
 '67270',
 '67400',
 '68040',
 '68590',
 '68610',
 '68710',
 '69160',
 '69630',
 '70500',
 '71050',
 '72450',
 '73390',
 '74630',


In [201]:
len(non_list)

167

In [202]:
non_list

['21830',
 '29820',
 '30570',
 '30700',
 '30730',
 '30850',
 '30890',
 '31130',
 '31570',
 '31680',
 '31820',
 '31950',
 '31960',
 '32450',
 '32520',
 '32710',
 '33280',
 '33500',
 '34170',
 '34790',
 '34860',
 '35690',
 '35800',
 '36070',
 '36320',
 '36390',
 '36640',
 '36720',
 '36780',
 '37530',
 '37600',
 '37650',
 '37930',
 '38080',
 '38470',
 '38540',
 '39030',
 '39090',
 '39140',
 '39220',
 '39230',
 '39280',
 '39940',
 '39990',
 '40420',
 '40630',
 '43850',
 '43870',
 '44360',
 '44780',
 '45630',
 '45640',
 '45840',
 '46610',
 '46710',
 '46860',
 '47220',
 '47650',
 '48000',
 '48190',
 '48830',
 '49670',
 '49870',
 '51220',
 '51940',
 '53100',
 '53430',
 '53550',
 '54400',
 '54450',
 '56030',
 '56580',
 '56600',
 '56980',
 '59070',
 '59560',
 '59660',
 '60310',
 '60350',
 '60470',
 '60540',
 '60630',
 '60670',
 '60900',
 '61030',
 '61450',
 '61460',
 '61570',
 '61820',
 '62930',
 '63340',
 '63610',
 '64180',
 '64280',
 '64800',
 '64940',
 '64980',
 '65500',
 '65510',
 '65620',


In [188]:
path3

'../new2/xbrl_df_cp932/18920/20170630421965_2.csv'

# memo

In [103]:
txt_path=path2+"/"+"{}".format(textlist[i])
with open (txt_path,'r') as t:
    pre_text=t.read()

path3=path2+"/"+"{}".format(acvlist[k])
df = pd.read_csv(path3, index_col=0,encoding="cp932")
df=recreate_df(df)
df=re_index(df)
flag=True
df=create_number(df,acvlist[i])
df = create_predict(df,pre_text,flag)
df

TypeError: can only concatenate str (not "float") to str

In [106]:
len(df.columns)

8

In [69]:
None_list=[None for i in range(0,len(df.columns))]
column = [df.columns[i] for i in range(0,len(df.columns))]
Nodf= pd.DataFrame([None_list],index=["通期"],columns=column)

In [79]:
Nodf=create_number(Nodf,acvlist[i])

In [80]:
Nodf

,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する当期純利益百万円,親会社株主に帰属する当期純利益％,number,prediction,pre_term
通期,None,None,None,None,None,None,None,None,20170710428574,None,None


In [94]:
for i in range(0,len(acvlist)):
    txt_path=path2+"/"+"{}".format(textlist[i])
    with open (txt_path,'r') as t:
        pre_text=t.read()
    if "csv" in acvlist[i]:
        path3=path2+"/"+"{}".format(acvlist[i])
        df = pd.read_csv(path3, index_col=0,encoding="cp932")
        df=recreate_df(df)
        df=re_index(df)
        flag=True
        df=create_number(df,acvlist[i])
        df = create_predict(df,pre_text,flag)
        
    else:
        flag=False
        None_list=[None for i in range(0,len(df.columns))]
        column = [df.columns[i] for i in range(0,len(df.columns))]
        df= pd.DataFrame([None_list],index=["通期"],columns=column)
        df=create_number(df,acvlist[i])
        df = create_predict(df,pre_text,flag)
    
    if i==0:
        newdf = df
    else:
        newdf = pd.concat([newdf,df])

In [95]:
newdf

,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する当期純利益百万円,親会社株主に帰属する当期純利益％,number,prediction,pre_term
第2四半期(累計),40000,-4.0,800,-66.5,750,-64.8,550,-71.3,20170710428574,True,平成30年2月期の連結業績予想
通期,79000,0.4,1500,-2.2,1400,-4.0,1000,-20.8,20170710428574,True,平成30年2月期の連結業績予想
通期,79000,0.4,1500,-2.2,1400,-4.0,1000,-20.8,20171005486026,True,平成30年2月期の連結業績予想
通期,79000,0.4,1500,-2.2,1400,-4.0,1000,-20.8,20171222441071,True,平成30年2月期の連結業績予想
第2四半期(累計),39600,0.1,1800,5.8,1700,13.5,1300,-4.4,20180413410788,True,平成31年2月期の連結業績予想
通期,79000,-0.1,2200,10.4,2000,7.7,1500,21.3,20180413410788,True,平成31年2月期の連結業績予想
第2四半期(累計),42000,6.2,3100,82.3,3100,106.9,2400,76.4,20180711480035,True,平成31年2月期の連結業績予想
通期,82000,3.7,4000,100.8,4000,115.4,3000,142.6,20180711480035,True,平成31年2月期の連結業績予想
通期,82000,3.7,5000,151.0,5000,169.2,4000,223.4,20181011416894,True,平成31年2月期の連結業績予想
通期,82000,3.7,5000,151.0,5000,169.2,4000,223.4,20190111459223,True,平成31年2月期の連結業績予想


In [1]:
# 以下純利益などの表

In [288]:
path='../new2/xbrl_asset'

In [289]:
col_dic={
    "総資産":"TotalAssets",
    "純資産":"NetAssets",
    "自己資本比率":"CapitalAdequacyRatio"
}

In [290]:
# xbrl_dfフォルダの中身の名前を取得
folderlist = os.listdir(path)
com_list=[]
for i in folderlist:
    if len(i)==5:
        com_list.append(i)

In [291]:
folderlist=com_list
folderlist=sorted(folderlist)

In [292]:
# xbrl_df/13010の中身のファイル名を取得
path2=path+"/"+"{}".format(folderlist[0])
filelist = os.listdir(path2)
filelist=sorted(filelist)

In [293]:
# 業績である_1だけを抽出
acvlist = []
for i in range(len(filelist)):
    if 'csv' in filelist[i]:
        acvlist.append(filelist[i])
acvlist=sorted(acvlist)
acvlist

['20170619410034.csv',
 '20170905468428.csv',
 '20171215436591.csv',
 '20180402403415.csv',
 '20180627471686.csv',
 '20180921409376.csv',
 '20190125463477.csv',
 '20190510420409.csv',
 '20190723474600.csv',
 '20191021410184.csv',
 '20200128451449.csv',
 '20200424499259.csv',
 '20200727467161.csv',
 '20201023407472.csv',
 '20210125447951.csv',
 '20210423498870.csv',
 '20210708463685.csv',
 '20211019413422.csv',
 '20220119569653.csv',
 '20220415522807.csv']

In [299]:
err_list=[]
for com in tqdm.tqdm(folderlist):
    # xbrl_df/13010の中身のファイル名を取得
    path2=path+"/"+"{}".format(com)
    filelist = os.listdir(path2)
    filelist=sorted(filelist)
    
    # 業績である_1だけを抽出
    acvlist = []
    for i in range(len(filelist)):
        if 'csv' in filelist[i]:
            acvlist.append(filelist[i])
    acvlist=sorted(acvlist)
    
    for i in range(0,len(acvlist)):

        try:  
            path3=path2+"/"+"{}".format(acvlist[i])
            df = pd.read_csv(path3, index_col=0)
            df=recreate_df(df)
            df=re_index(df)
            df=create_number(df,acvlist[i])
            #今季の結果のdataframe
            adf = df.drop(index=df.index[[1]])
            #前期の結果のdataframe
            bdf = df.drop(index=df.index[[0]])
            if i==0:
                thisdf = adf
                predf = bdf
                continue
            else:
                thisdf = pd.concat([thisdf,adf])
                predf = pd.concat([predf,bdf])

            #thisdf.to_csv("../new2/df_sum/asset_this/{}.csv".format(com),encoding="cp932")
            #predf.to_csv("../new2/df_sum/asset_pre/{}.csv".format(com),encoding="cp932")


        except:
            err_list.append(com)

100%|███████████████████████████████████████| 2958/2958 [01:23<00:00, 35.32it/s]


In [295]:
df

,総資産百万円,総資産百万円,純資産％,純資産円銭,number
2019年12月期,8624,2724,31.6,745.58,20200213463368
2018年12月期,8587,1999,23.3,550.50,20200213463368


In [297]:
thisdf

,総資産百万円,純資産百万円,自己資本比率％,number
30年3月期第1四半期,5724,1366,23.9,20170808451410
30年3月期第2四半期,6121,1516,24.8,20171102407026
30年3月期第3四半期,6564,1589,24.2,20180202462463
30年3月期,7494,1715,22.9,20180508429378
30年3月期,7494,1715,22.9,20180529450232
30年12月期第1四半期,7515,1791,23.8,20180809495383
30年12月期第2四半期,7935,1907,24.0,20181113436441
平成30年12月期,8587,1999,23.3,20190213475717
2019年12月期第1四半期,8859,2306,26.0,20190510420476
2019年12月期第2四半期,7764,2297,29.6,20190808485118


In [37]:
err_list

['20200213463368.csv',
 '20210210461632.csv',
 '20220214588928.csv',
 '20190513424390.csv',
 '20201105416794.csv',
 '20190510420533.csv',
 '20210812484564.csv',
 '20211110429749.csv',
 '20220209584183.csv',
 '20211103423827.csv',
 '20190206471170.csv',
 '20220512543938.csv']

In [39]:
err_list

['17240',
 '17240',
 '17240',
 '22010',
 '24890',
 '34070',
 '65390',
 '65390',
 '65390',
 '88500',
 '90310',
 '93100']

# errの処理


In [285]:
path='../new2/xbrl_asset/17240/20200213463368.csv'

In [286]:
df = pd.read_csv(path, index_col=0)

In [287]:
df

,0,1,2,3
None,百万円,百万円,％,円銭
2019年12月期,8624,2724,31.6,745.58
2018年12月期,8587,1999,23.3,550.50
index,総資産,総資産,純資産,純資産


In [300]:
err_list

['17240',
 '17240',
 '17240',
 '22010',
 '24890',
 '34070',
 '65390',
 '65390',
 '65390',
 '88500',
 '90310',
 '93100']

In [316]:
err_list=list(set(err_list))
err_list

['17240', '22010', '24890', '34070', '93100', '90310', '88500', '65390']

In [339]:
err_list=['17240', '22010', '24890', '34070', '93100', '90310', '88500', '65390']

In [386]:
for com in tqdm.tqdm(err_list):
    # xbrl_df/13010の中身のファイル名を取得
    path2=path+"/"+"{}".format(com)
    filelist = os.listdir(path2)
    filelist=sorted(filelist)
    
    # 業績である_1だけを抽出
    acvlist = []
    for i in range(len(filelist)):
        if 'csv' in filelist[i]:
            acvlist.append(filelist[i])
    acvlist=sorted(acvlist)
    
    for i in range(0,len(acvlist)):
        try:
            
            path3=path2+"/"+"{}".format(acvlist[i])
            df = pd.read_csv(path3, index_col=0)
            df=recreate_df(df)
            df=re_index(df)
            df=create_number(df,acvlist[i])
            #今季の結果のdataframe
            adf = df.drop(index=df.index[[1]])
            #前期の結果のdataframe
            bdf = df.drop(index=df.index[[0]])
            if i==0:
                thisdf = adf
                predf = bdf
                continue
            else:
                thisdf = pd.concat([thisdf,adf])
                predf = pd.concat([predf,bdf])
        
        except:
            if len(df.columns)==len(thisdf.columns):
                df.columns=list(thisdf.columns)
            elif len(df.columns)==5:
                df.columns=['総資産百万円', '純資産百万円', '自己資本比率％',"1株当たり純資産円銭","number"]
                
            elif len(df.columns)==4:
                df.columns=['総資産百万円', '純資産百万円', '自己資本比率％',"number"]
            #今季の結果のdataframe
            adf = df.drop(index=df.index[[1]])
            #前期の結果のdataframe
            bdf = df.drop(index=df.index[[0]])
            thisdf = pd.concat([thisdf,adf])
            predf = pd.concat([predf,bdf])
            
            
        thisdf.to_csv("../new2/df_sum/err/asset_this/{}.csv".format(com),encoding="cp932")
        predf.to_csv("../new2/df_sum/err/asset_pre/{}.csv".format(com),encoding="cp932")

 88%|███████████████████████████████████████▍     | 7/8 [00:00<00:00, 21.16it/s]


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [306]:
df.columns=['総資産百万円', '純資産百万円', '自己資本比率％',"1株当たり純資産円銭","number"]

In [345]:
df

,総資産百万円,純資産百万円,自己資本比率％,number
2022年3月期第1四半期,9580,8952,93.2,20210812484564
2021年3月期,10158,9380,92.1,20210812484564


In [332]:
len(thisdf.columns)

4

In [392]:
com=65390
path2=path+"/"+"{}".format(com)
filelist = os.listdir(path2)
filelist=sorted(filelist)

# 業績である_1だけを抽出
acvlist = []
for i in range(len(filelist)):
    if 'csv' in filelist[i]:
        acvlist.append(filelist[i])
acvlist=sorted(acvlist)

for i in range(0,len(acvlist)):

        path3=path2+"/"+"{}".format(acvlist[i])
        df = pd.read_csv(path3, index_col=0)
        df=recreate_df(df)
        df=re_index(df)
        df=create_number(df,acvlist[i])
        if len(df.columns)==5:
            df.columns=['総資産百万円', '純資産百万円', '自己資本比率％',"1株当たり純資産円銭","number"]
        #今季の結果のdataframe
        adf = df.drop(index=df.index[[1]])
        #前期の結果のdataframe
        bdf = df.drop(index=df.index[[0]])
        if i==0:
            thisdf = adf
            predf = bdf
            continue
        else:
            thisdf = pd.concat([thisdf,adf])
            predf = pd.concat([predf,bdf])

thisdf.to_csv("../new2/df_sum/err/asset_this/{}.csv".format(com),encoding="cp932")
predf.to_csv("../new2/df_sum/err/asset_pre/{}.csv".format(com),encoding="cp932")

In [391]:
 thisdf

,総資産百万円,純資産百万円,自己資本比率％,1株当たり純資産円銭,number
2021年3月期,10158,9380,92.1,374.65,20210512412922
2022年3月期第1四半期,9580,8952,93.2,NaN,20210812484564
2022年3月期第2四半期,9893,9130,92.1,NaN,20211110429749
2022年3月期第3四半期,9902,9325,94.0,NaN,20220209584183
2022年3月期,10412,9596,92.0,383.34,20220511541079


In [389]:
thisdf

,総資産百万円,総資産百万円,純資産％,純資産円銭,number
2021年3月期,10158,9380,92.1,374.65,20210512412922


In [383]:
path3

'../new2/xbrl_asset/24890/20201105416794.csv'

In [378]:
kdf=df

In [380]:
kdf.index=['None', '2021年3月期第2四半期', '2020年3月期', 'index']

In [367]:
kdf=kdf.reindex(index=index)

In [381]:
kdf

,0,1,2,3
None,百万円,百万円,％,円銭
2021年3月期第2四半期,20474,12991,62.4,307.42
2020年3月期,18986,12448,64.6,294.9
index,総資産,純資産,自己資本比率,1株当たり純資産


In [384]:
kdf.to_csv("../new2/xbrl_asset/24890/20201105416794.csv")

In [ ]:
'17240',
 '17240',
 '17240',
 '22010',
 '24890',
 '34070',
 '65390',
 '65390',
 '65390',
 '88500',